In [27]:
# Importamos las librerías que necesitamos

# Manejo del sistema
# -----------------------------------------------------------------------
import sys
sys.path.append("../")  # Agregamos el directorio principal al sistema de rutas

# Barra de progreso
# -----------------------------------------------------------------------
from tqdm import tqdm

# Funciones personalizadas
# -----------------------------------------------------------------------
from src import sp_imdb as si

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd

# Conexión a base de datos
# -----------------------------------------------------------------------
import pymongo

# Manejo de advertencias
# -----------------------------------------------------------------------
import warnings


In [28]:
# lo primero que tenemos que hacer es conectarnos con Mongo
cliente = pymongo.MongoClient()
# nos conectamos con una BBDD 
imdb = cliente['IMDB']
peliculas = imdb['Peliculas']

In [29]:
df = pd.read_csv("../data/api_generos.csv")

In [35]:
si.obtener_pelis(df.loc[1::10,'id'])

 66%|██████▌   | 20768/31458 [2:13:15<1:08:35,  2.60it/s] 


InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x0000000104d984dc chromedriver + 4162780
1   chromedriver                        0x0000000104d90664 chromedriver + 4130404
2   chromedriver                        0x00000001049e79f8 chromedriver + 293368
3   chromedriver                        0x0000000104a215ec chromedriver + 529900
4   chromedriver                        0x0000000104a22930 chromedriver + 534832
5   chromedriver                        0x0000000104d5de08 chromedriver + 3923464
6   chromedriver                        0x0000000104d623dc chromedriver + 3941340
7   chromedriver                        0x0000000104d46038 chromedriver + 3825720
8   chromedriver                        0x0000000104d62f3c chromedriver + 3944252
9   chromedriver                        0x0000000104d386f4 chromedriver + 3770100
10  chromedriver                        0x0000000104d7f980 chromedriver + 4061568
11  chromedriver                  

In [46]:
si.obtener_actores()

100%|██████████| 19796/19796 [00:00<00:00, 4202796.21it/s]


In [11]:
pelis_eliminar = list(peliculas.find({"$and":[{'puntuacion':float('nan')}, {'duracion':float('nan')}, {'argumento':float('nan')}]}))

In [9]:
for peli in pelis_eliminar:
    peliculas.delete_one({"_id": peli['_id']})

In [18]:
if nan_docs_ids:
    peliculas.delete_many({"_id": {"$in": nan_docs_ids}})
    print(f"Se eliminaron {len(nan_docs_ids)} documentos con NaN.")
else:
    print("No se encontraron documentos con NaN.")

{'puntuacion': 8.4,
 'direccion': ['Anthony Russo', 'Joe Russo'],
 'guion': ['Christopher Markus', 'Stephen McFeely', 'Stan Lee'],
 'duracion': 181,
 'argumento': nan}